<font color=lightblue size=20>Projeto Telecom X: Análise de evasão de clientes

# 📌 Extração

- **Importação:** Optei por usar a url do arquivo raw do GitHub a fim de facilitar a utilização do notebook por um usuário com poucos conhecimentos de importação de dados, visto que dessa forma seria necessário apenas executar o código ao invés de precisar importar o arquivo com os dados. Para tal fim foi necessário usar a biblioteca `requests` para buscar os dados e usar `json.loads()` para carregar os dados corretamente.

- **Normalização:** Como haviam colunas aninhadas (customer, phone, internet, account) utilizei `json_normalize()` para expandir as colunas.

Os dados foram salvos num DataFrame nomeado `df`.

## Código

In [13]:
import pandas as pd
import numpy as np
import requests
import json

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

response = requests.get(url) # Requests usado para carregar os dados json
response.raise_for_status() # Lança uma exceção para códigos de status HTTP ruins (4xx ou 5xx)

# Carrega o conteúdo JSON como um objeto Python
dados_json = json.loads(response.text)

# Normaliza os dados
df_normalizado = pd.json_normalize(dados_json, sep='_')

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Monthly,account_Charges_Total
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65
7266,9995-HOTOH,No,Male,0,Yes,Yes,63,No,No phone service,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Electronic check,59.00,3707.6


## Exibição dos dados

In [21]:
df_normalizado.head()

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Monthly,account_Charges_Total
0,0002-orfbo,0,female,0,1,1,9,1,no,dsl,...,yes,no,yes,yes,no,one year,1,mailed check,65.6,593.30
1,0003-mknfe,0,male,0,0,0,9,1,yes,dsl,...,no,no,no,no,yes,month-to-month,0,mailed check,59.9,542.40
2,0004-tlhlj,1,male,0,0,0,4,1,no,fiber optic,...,no,yes,no,no,no,month-to-month,1,electronic check,73.9,280.85
3,0011-igkff,1,male,1,1,0,13,1,no,fiber optic,...,yes,yes,no,yes,yes,month-to-month,1,electronic check,98.0,1237.85
4,0013-exchz,1,female,1,1,0,3,1,no,fiber optic,...,no,no,yes,yes,no,month-to-month,1,mailed check,83.9,267.40


#🔧 Transformação

- **Duplicados:** Não encontrei dados duplicados.

- **nan e null:** Não encontrei nos dados.

- **Conversão de tipos:** Ao ferificar os tipos de dados percebi que a coluna com os gastos totais de cada cliente estava em `object`, por isso fiz a conversão para `np.float64`. Para isso precisei alterar strings vazias para 0.

- **Strings:** Padronizei os textos para letras minúsculas. Para isso criei uma variável somente com os nomes das colunas com texto e iterei sobre `df` utilizando essa variável como referência transformando em minúscula coluna por coluna.

- **Colunas binárias:** Nas colunas que apresentavam apenas dois valores de sim e não transformei sim em 1 e não em 0.

- **Tradução:** Os nomes das colunas bem como os dados em inglês foram traduzidos para o português a fim de facilitar a vizualização e entendimento dos dados.

- **Simplicação:** Na coluna `multiplas_linhas` o termo 'No phone service' foi simplificado para 'na' de não se aplica. O mesmo foi feito nas colunas `protecao_online, backup_online, protecao_dispositivo, suporte_tecnico, tv_streaming, filmes_streaming:` com o termo 'No internet service'.

- **Remoção da coluna `id`:** Reemovi a coluna `id` pois não é relevante para a análise.

- **Coluna `Contas_Diarias`:** Criação da coluna `Contas_Diarias`, dividindo a coluna `cobranca_mensal` por 30 dias, para uma visão mais detalhada do comportamento dos clientes ao longo do tempo.

- **round(2):** Deixei as colunas com tipo float com duas casas decimais.

- **Dicionário de dados:** No final deste notebook vai encontrar um dicionário de dados com uma breve explicação das informações contidas em cada coluna.

## Código

In [14]:
# ANALISANDO OS DADOS
print(f'TOTAL DE DUPLICADOS: {df_normalizado.duplicated().sum()}') # Conferindo duplicados
for item in df_normalizado.columns: # verificando as colunas
   print(f'{item:>30}: {df_normalizado[item].dtype} => NA: {df_normalizado[item].isna().sum()} '
   f'=> NULL: {df_normalizado[item].isnull().sum()} => DADOS: {pd.unique(df_normalizado[item])}')

TOTAL DE DUPLICADOS: 0
                    customerID: object => NA: 0 => NULL: 0 => DADOS: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
                         Churn: object => NA: 0 => NULL: 0 => DADOS: ['No' 'Yes' '']
               customer_gender: object => NA: 0 => NULL: 0 => DADOS: ['Female' 'Male']
        customer_SeniorCitizen: int64 => NA: 0 => NULL: 0 => DADOS: [0 1]
              customer_Partner: object => NA: 0 => NULL: 0 => DADOS: ['Yes' 'No']
           customer_Dependents: object => NA: 0 => NULL: 0 => DADOS: ['Yes' 'No']
               customer_tenure: int64 => NA: 0 => NULL: 0 => DADOS: [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
            phone_PhoneService: object => NA: 0 => NULL: 0 => DADOS: ['Yes' 'No']
           phone_MultipleLines: object 

In [15]:
# ALTERANDO COLUNA DE GASTOS TOTAIS PARA FLOAT
# Substituindo strings vazias por NAN
df_normalizado['account_Charges_Total'] = df_normalizado['account_Charges_Total'].replace(' ', 0)
# Alterando o tipo de dados da coluna de str para float
df_normalizado['account_Charges_Total'] = df_normalizado['account_Charges_Total'].astype(np.float64)

In [16]:
# MODIFICANDO COLUNAS COM STRINGS PARA LOWER CAPS
# Separando colunas com string
colunas_tratar = df_normalizado.columns
colunas_tratar = colunas_tratar.drop(['account_Charges_Monthly', 'account_Charges_Total',
                        'customer_SeniorCitizen', 'customer_tenure'])
# aplicando lower no nas colunas com string
for column in df_normalizado.columns:
  if column in colunas_tratar:
    df_normalizado[column] = df_normalizado[column].str.lower()

In [17]:
# CONVERTENDO STRINGS DE YES E NO EM 1 E 0
# Substituindo strings vazias por no na coluna churn
df_normalizado['Churn'] = df_normalizado['Churn'].replace(r'^\s*$', 'no', regex=True)
# Pegando colunas com apenas yes e no
colunas_converter = ['Churn', 'customer_Partner', 'customer_Dependents',
                  'phone_PhoneService', 'account_PaperlessBilling']
# Substituindo yes e no por 1 e 0
for column in df_normalizado.columns:
  if column in colunas_converter:
    df_normalizado[column] = df_normalizado[column].map({'yes': 1, 'no':0})

In [18]:
# RENOMEANDO AS COLUNAS PARA PT-BR
# Criar um dicionário de mapeamento para renomear as colunas
novos_nomes_colunas = {
    'customerID': 'id_cliente',
    'Churn': 'evasao',
    'customer_gender': 'genero',
    'customer_SeniorCitizen': 'idoso',
    'customer_Partner': 'parceiro',
    'customer_Dependents': 'dependentes',
    'customer_tenure': 'meses_contrato',
    'phone_PhoneService': 'servico_telefone',
    'phone_MultipleLines': 'multiplas_linhas',
    'internet_InternetService': 'servico_internet',
    'internet_OnlineSecurity': 'protecao_online',
    'internet_OnlineBackup': 'backup_online',
    'internet_DeviceProtection': 'protecao_dispositivo',
    'internet_TechSupport': 'suporte_tecnico',
    'internet_StreamingTV': 'tv_streaming',
    'internet_StreamingMovies': 'filmes_streaming',
    'account_Contract': 'tipo_contrato',
    'account_PaperlessBilling': 'fatura_digital',
    'account_PaymentMethod': 'metodo_pagamento',
    'account_Charges_Monthly': 'cobranca_mensal',
    'account_Charges_Total': 'cobranca_total'
}
# Recomenado as colunas
df_traduzido = df_normalizado.rename(columns=novos_nomes_colunas)

In [19]:
# TRADUZINDO TERMOS DO EN-US PARA PT-BR
# Colunas para traduzir
colunas_traduzir = [ 'genero', 'multiplas_linhas', 'servico_internet',
                    'protecao_online', 'backup_online', 'protecao_dispositivo',
                     'suporte_tecnico', 'tv_streaming', 'filmes_streaming',
                     'tipo_contrato', 'metodo_pagamento']
# Dicionário com as palavras para traduzir
traducao = {'female': 'mulher', 'male': 'homem',
            'no': 'nao',
            'yes': 'sim',
            'fiber optic': 'fibra otica',
            'no phone service': 'na',
            'no internet service': 'na',
            'one year': 'anual',
            'month-to-month':'cada mes',
            'two year': 'bienal',
            'mailed check': 'cheque correios',
            'electronic check': 'cheque eletronico',
            'credit card (automatic)': 'cartao credito automatico',
            'bank transfer (automatic)': 'transferencia automatica'}
# executando a traduçao nas colunas
for column in df_traduzido.columns:
  if column in colunas_traduzir:
    df_traduzido[column] = df_traduzido[column].replace(traducao)

In [24]:
# DROPANDO A COLUNA ID
df_clientes = df_traduzido.drop(columns='id_cliente')

In [44]:
# ARREDONDANDO A COLIUNA cobranca_mensal PARA 2 CASAS DECIMAIS
df_clientes['cobranca_mensal'] = df_clientes['cobranca_mensal'].round(2)

In [45]:
# CRIANDO A COLUNA Contas_Diarias
df_clientes['contas_diarias'] = (df_clientes['cobranca_mensal'] / 30).round(2)

In [46]:
# CONFERINDO OS DADOS
print(f'TOTAL DE DUPLICADOS: {df_clientes.duplicated().sum()}') # Conferindo duplicados
for item in df_clientes.columns: # verificando as colunas
   print(f'{item:>30}: {df_clientes[item].dtype} => NA: {df_clientes[item].isna().sum()} '
   f'=> NULL: {df_clientes[item].isnull().sum()} => DADOS: {pd.unique(df_clientes[item])}')

TOTAL DE DUPLICADOS: 179
                        evasao: int64 => NA: 0 => NULL: 0 => DADOS: [0 1]
                        genero: object => NA: 0 => NULL: 0 => DADOS: ['mulher' 'homem']
                         idoso: int64 => NA: 0 => NULL: 0 => DADOS: [0 1]
                      parceiro: int64 => NA: 0 => NULL: 0 => DADOS: [1 0]
                   dependentes: int64 => NA: 0 => NULL: 0 => DADOS: [1 0]
                meses_contrato: int64 => NA: 0 => NULL: 0 => DADOS: [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
              servico_telefone: int64 => NA: 0 => NULL: 0 => DADOS: [1 0]
              multiplas_linhas: object => NA: 0 => NULL: 0 => DADOS: ['nao' 'sim' 'na']
              servico_internet: object => NA: 0 => NULL: 0 => DADOS: ['dsl' 'fibra otica' 'nao']
               protecao_online: object =

## Exibião dos dados

In [43]:
df_clientes.head()

,evasao,genero,idoso,parceiro,dependentes,meses_contrato,servico_telefone,multiplas_linhas,servico_internet,protecao_online,...,protecao_dispositivo,suporte_tecnico,tv_streaming,filmes_streaming,tipo_contrato,fatura_digital,metodo_pagamento,cobranca_mensal,cobranca_total,contas_diarias
0,0,mulher,0,1,1,9,1,nao,dsl,nao,...,nao,sim,sim,nao,anual,1,cheque correios,65.6,593.3,2.19
1,0,homem,0,0,0,9,1,sim,dsl,nao,...,nao,nao,nao,sim,cada mes,0,cheque correios,59.9,542.4,2.00
2,1,homem,0,0,0,4,1,nao,fibra otica,nao,...,sim,nao,nao,nao,cada mes,1,cheque eletronico,73.9,280.8,2.46
3,1,homem,1,1,0,13,1,nao,fibra otica,nao,...,sim,nao,sim,sim,cada mes,1,cheque eletronico,98.0,1237.8,3.27
4,1,mulher,1,1,0,3,1,nao,fibra otica,nao,...,nao,sim,sim,nao,cada mes,1,cheque correios,83.9,267.4,2.80


#📊 Carga e análise

#📄Relatorio Final

## **Dicionário de dados**

* **id_cliente**: número de identificação único de cada cliente (String)
* **evasao**: se o cliente deixou ou não a empresa. (int64 / 0 = não e 1 = sim)
* **genero**: gênero (masculino e feminino)
* **senior_citizen**: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
* **parceiro**: se o cliente possui ou não um parceiro ou parceira
* **dependentes**: se o cliente possui ou não dependentes
* **meses_contrato**: meses de contrato do cliente
* **servico_telefone**: assinatura de serviço telefônico
* **multiplas_linhas**: assinatura de mais de uma linha de telefone (String / 'nao', 'sim' e 'na' para não se aplica)
* **servico_internet**: assinatura de um provedor internet
* **backup_online**: assinatura adicional de backup online
* **protecao_dispositivo**: assinatura adicional de proteção no dispositivo
* **suporte_tecnico**: assinatura adicional de suporte técnico, menos tempo de espera
* **tv_streaming**: assinatura de TV a cabo
* **filmes_streaming**: assinatura de streaming de filmes
* **tipo_contrato**: tipo de contrato
* **fatura_digital**: se o cliente prefere receber online a fatura
* **metodo_pagamento**: forma de pagamento
* **cobranca_mensal**: total de todos os serviços do cliente por mês
* **cobranca_total**: total gasto pelo cliente